<a href="https://colab.research.google.com/github/12NaN/data-science-project/blob/master/modifiedTrainData_RF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from google.colab import files
from sklearn.ensemble import RandomForestRegressor , GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor 
from sklearn.model_selection import KFold, train_test_split
import xgboost as xgb
%matplotlib inline

In [0]:
train_df = pd.read_csv('https://grantmlong.com/data/SE_rents2018_train.csv', index_col=0)
test_df = pd.read_csv('https://grantmlong.com/data/SE_rents2018_test1.csv', index_col=0)
submit1_df = pd.read_csv('https://grantmlong.com/data/SE_rents2018_test2.csv', index_col=0)
submit2_df = pd.read_csv('https://grantmlong.com/data/SE_rents2018_test3.csv', index_col=0)
schools_df = pd.read_csv('https://data.cityofnewyork.us/resource/23z9-6uk9.csv', usecols = ['postcode'])

#train_df = train_df.drop(train_df[(train_df['size_sqft']>8000)& (train_df['rent']<10000)].index).reset_index(drop=True)

#train_df = train_df.drop(train_df[(train_df['bathrooms']>=12) 
                       #  & (train_df['rent']<10000)].index).reset_index(drop=True)

#train_df = train_df.drop(train_df[(train_df['bathrooms']==6) 
                       #  & (train_df['rent']<20000)].index).reset_index(drop=True)


#train_df = train_df.drop(train_df[(train_df['bedrooms']>=8) 
                        # & (train_df['rent']<20000)].index).reset_index(drop=True)

#train_df = pd.merge(train_df,schools_df,left_on="addr_zip", right_on="postcode")
#cf = submit2_df #retaining the dataframe in cf instead incase I want to look at the original without edits
#cf.min_to_subway = cf.min_to_subway.fillna(cf.min_to_subway.mean(), axis=0) #Replacing empty values with the mean of the entire column
#cf.year_built = cf.year_built.fillna(cf.year_built.median(), axis=0) #Replacing empty values with the median of the entire column

In [0]:

# Append test1 to train_df
train_df = train_df.append(test_df, ignore_index = True)

In [404]:
cols_with_null = train_df.columns[train_df.isna().any()] # Return columns with null values 

train_df[cols_with_null].isna().sum().sort_values(ascending=False) # Number of null values

line             4135
floornumber      1502
year_built        470
min_to_subway     143
unit              114
addr_unit         110
description        16
neighborhood        3
bin                 1
dtype: int64

In [0]:
#missing data for  df_train

cf = train_df #retaining the dataframe in cf instead incase I want to look at the original without edits
cf.min_to_subway = cf.min_to_subway.fillna(cf.min_to_subway.mean(), axis=0) #Replacing empty values with the mean of the entire column
cf.year_built = cf.year_built.fillna(cf.year_built.mean(), axis=0) #Replacing empty values with the median of the entire column
cf.floor_count = cf.floor_count.fillna(cf.floor_count.mean(), axis=0)
cf.floornumber = cf.floornumber.fillna(cf.floornumber.mean(), axis=0)
cf.size_sqft = cf.size_sqft.fillna(cf.size_sqft.mean(), axis=0)


In [406]:
pd.isnull(train_df).sum() > 0

addr_unit                  True
building_id               False
bedrooms                  False
bathrooms                 False
size_sqft                 False
created_at                False
addr_street               False
addr_city                 False
addr_zip                  False
addr_lat                  False
addr_lon                  False
bin                        True
bbl                       False
floor_count               False
year_built                False
min_to_subway             False
has_doorman               False
has_elevator              False
has_fireplace             False
has_dishwasher            False
is_furnished              False
has_gym                   False
allows_pets               False
has_washer_dryer          False
has_garage                False
has_roofdeck              False
has_concierge             False
has_pool                  False
has_garden                False
has_childrens_playroom    False
rent                      False
no_fee  

In [407]:
train_df.sample(5).transpose()

,651,5413,10967,6025,11697
addr_unit,#3,#3,#5B,#4A,#3
building_id,1305769,34501,49807,88069,379990
bedrooms,2,1,2,3,2
bathrooms,1,1,2,1,1
size_sqft,800,2400,1190,1100,800
created_at,2018-05-09 15:11:36,2018-06-02 12:23:43,2018-05-08 14:12:51,2018-07-13 21:18:40,2018-05-22 20:36:55
addr_street,80-21 ASTORIA BOULEVARD,529 9 AVENUE,308 EAST 38 STREET,1510 YORK AVENUE,858 METROPOLITAN AVENUE
addr_city,East Elmhurst,New York,New York,New York,Brooklyn
addr_zip,11370,10018,10016,10075,11211
addr_lat,40.7655,40.7569,40.7468,40.7718,40.7144


Build training data

In [0]:
def get_cv_results(classifier, train_df):
    results = []
    k_fold = KFold(n_splits=10)
    
    for train, test in k_fold.split(train_df):
        classifier.fit(train_df.loc[train, feature_cols], train_df.loc[train, 'rent'])
        y_predicted = classifier.predict(train_df.loc[test, feature_cols])
        accuracy = mean_squared_error(train_df.loc[test, 'rent'], y_predicted)
        results.append(accuracy)
    return np.mean(results), np.std(results)

In [0]:
feature_cols = [
     'bedrooms', 'year_built', 'bathrooms', 'min_to_subway', 
    'size_sqft', 'no_fee', 'has_doorman'          
]
train_features = train_df[feature_cols + ['rent']] 
#train_features = total_df[feature_cols] 
train_features = pd.get_dummies(train_features)
# impute missing values with medians
feature_cols = train_features.columns.drop('rent')
#train_features = train_features.fillna(train_features.median(), axis=0)

# construct target vector
#train_target = train_df['rent']
#train_target = total_df['rent']

In [0]:
model_train_df, model_holdout_df = train_test_split(train_features, test_size=0.2)
model_train_df.reset_index(inplace=True)
model_holdout_df.reset_index(inplace=True)

Fit model

In [411]:
lreg = LinearRegression()
get_cv_results(lreg, model_train_df)



(3870896.557156579, 983507.9924381691)

In [412]:

rf = RandomForestRegressor()
get_cv_results(rf, model_train_df)



/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


(2229883.2940650866, 832265.2322679556)

In [413]:
DTR = DecisionTreeRegressor(random_state = 0)
get_cv_results(DTR, model_train_df)

(4111453.150933468, 1402223.3496656518)

In [414]:
gbr = GradientBoostingRegressor()
get_cv_results(gbr, model_train_df)


(2235965.897518594, 746687.1959656283)

In [415]:
xgbr = xgb.XGBRegressor()
get_cv_results(xgbr, model_train_df)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[07:26:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[07:26:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


...

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[07:27:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[07:27:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(2257748.181643103, 759774.0659399837)

In [416]:
#lreg = LinearRegression()
#lreg.fit(train_features, train_target)
#  lreg.fit(model_train_df[feature_cols], model_train_df['rent'])
#  predicted = lreg.predict(model_holdout_df[feature_cols])
 # mean_squared_error(model_holdout_df['rent'],predicted)

#rf = RandomForestRegressor()
#rf.fit(train_features, train_target)
rf.fit(model_train_df[feature_cols], model_train_df['rent'])
predicted = rf.predict(model_holdout_df[feature_cols])
mean_squared_error(model_holdout_df['rent'],predicted)


#DTR = DecisionTreeRegressor(random_state = 0)

#DTR.fit(train_features, train_target);

#gbr = GradientBoostingRegressor()
#gbr.fit(train_features, train_target)


#xgbr = xgb.XGBRegressor()
#xgbr.fit(train_features, train_target)

2501284.373309955

In [429]:
#missing data for  df_submit

cf2 = submit2_df #retaining the dataframe in cf instead incase I want to look at the original without edits
cf2.min_to_subway = cf2.min_to_subway.fillna(cf2.min_to_subway.mean(), axis=0) #Replacing empty values with the mean of the entire column
cf2.year_built = cf2.year_built.fillna(cf2.year_built.mean(), axis=0) #Replacing empty values with the median of the entire column
cf2.floor_count = cf2.floor_count.fillna(cf2.floor_count.mean(), axis=0)
cf2.floornumber = cf2.floornumber.fillna(cf2.floornumber.mean(), axis=0)
cf2.size_sqft = cf2.size_sqft.fillna(cf2.size_sqft.mean(), axis=0)
cf2.rent = cf2.fillna(lambda x: x.median())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [430]:
pd.isnull(submit2_df).sum() > 0

building_id      False
bedrooms         False
bathrooms        False
size_sqft        False
addr_zip         False
                 ...  
rent_18772111    False
rent_18772123    False
rent_18772126    False
rent_18772138    False
rent_18772162    False
Length: 8985, dtype: bool

In [0]:

submit2_df = pd.get_dummies(submit2_df)
rf.fit(train_features[feature_cols], train_features.get('rent'))
predicted = rf.predict(submit2_df[feature_cols])
#mean_squared_error(submit2_df.get('rent'),predicted)

Predict and Measure Using Test 1

In [0]:
'''
test_features = test_df[feature_cols] 

# impute missing values with medians
test_features = test_features.fillna(train_features.median(), axis=0)

# construct predictions 
test_df['predicted'] = lreg.predict(test_features)

mean_squared_error(test_df['rent'], test_df['predicted'])
'''

Combine Data, Predict Values for Test 3

In [0]:
'''
master_df = train_df.append(test_df, sort=False)
#master_df = total_df.append(test_df, sort=False)
master_features = master_df[feature_cols].fillna(master_df[feature_cols].median(), axis=0)
master_target = master_df['rent']

rf.fit(master_features, master_target)
'''

In [0]:
'''
train_df.corr() #Looking at which features correlate the most with the rent from the training data
#total_df.corr()
'''

Create Submission File for test2

In [0]:
'''
submit2_features = submit2_df[feature_cols].fillna(master_df[feature_cols].median(), axis=0)
submit2_df['predictions'] = rf.predict(submit2_features) 
submit2_df['predictions'].to_csv('sample_submission1.csv', header=True)

submit2_df['fake_rent'] = np.ones(submit2_df['predictions'].shape) * master_target.median()
mean_squared_error(submit2_df['predictions'], test_df['rent'])
'''

In [0]:
'''
test=submit2_df.sort_values(['predictions'],ascending=[1])
print(test)
'''

In [0]:
'''
submit2_features = submit2_df[feature_cols].fillna(master_df[feature_cols].median(), axis=0)
submit2_df['predictions'] = xgbr.predict(submit2_features) 
submit2_df['predictions'].to_csv('sample_submission1.csv', header=True)

submit2_df['fake_rent'] = np.ones(submit2_df['predictions'].shape) * master_target.median()
mean_squared_error(submit2_df['predictions'], test_df['rent'])
'''

In [0]:
'''
test=submit2_df.sort_values(['predictions'],ascending=[1])
print(test)
'''

In [0]:
'''
submit2_features = submit2_df[feature_cols].fillna(master_df[feature_cols].median(), axis=0)
submit2_df['predictions'] = gbr.predict(submit2_features) 
submit2_df['predictions'].to_csv('sample_submission1.csv', header=True)

submit2_df['fake_rent'] = np.ones(submit2_df['predictions'].shape) * master_target.median()
mean_squared_error(submit2_df['predictions'], test_df['rent'])
'''

In [0]:
'''
test=submit2_df.sort_values(['predictions'],ascending=[1])
print(test)
'''

In [0]:
'''
submit2_features = submit2_df[feature_cols].fillna(master_df[feature_cols].median(), axis=0)
submit2_df['predictions'] = lreg.predict(submit2_features) 
submit2_df['predictions'].to_csv('sample_submission1.csv', header=True)

submit2_df['fake_rent'] = np.ones(submit2_df['predictions'].shape) * master_target.median()
mean_squared_error(submit2_df['predictions'], test_df['rent'])
'''

In [0]:
'''
test=submit2_df.sort_values(['predictions'],ascending=[1])
print(test)
'''

In [0]:
''''
submit2_features = submit2_df[feature_cols].fillna(master_df[feature_cols].median(), axis=0)
submit2_df['predictions'] = DTR.predict(submit2_features) 
submit2_df['predictions'].to_csv('sample_submission1.csv', header=True)

submit2_df['fake_rent'] = np.ones(submit2_df['predictions'].shape) * master_target.median()
mean_squared_error(submit2_df['predictions'], test_df['rent'])
'''

In [0]:
'''
test=submit2_df.sort_values(['predictions'],ascending=[1])
print(test)
'''

In [0]:
#files.download("sample_submission1.csv")

In [0]:
test['predictions'].hist() #Confirming largest predicted rent is 15516 visually. As expected we see that there are more apartments ranging from 2k-3k than any other price range.

In [0]:
submit2_df.isna().sum()